# 1.1. Get the list of animes   
We start from the list of animes to include in your corpus of documents. In particular, we focus on the top animes ever list. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).

The output of this step is a .txt file whose single line corresponds to an anime's url.

#1.2. Crawl animes    
Once you get all the urls in the first 400 pages of the list, you:

Download the html corresponding to each of the collected urls.
After you collect a single page, immediately save its html in a file. In this way, if your program stops, for any reason, you will not lose the data collected up to the stopping point. More details in Important (2).
Organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.


In [ ]:
!git clone https://github.com/S4b3/ADM-HW3-Group1.git

Cloning into 'ADM-HW3-Group1'...
remote: Enumerating objects: 153084, done.
remote: Counting objects: 100% (153084/153084), done.
remote: Compressing objects: 100% (118228/118228), done.
remote: Total 153084 (delta 104460), reused 83465 (delta 34850), pack-reused 0
Receiving objects: 100% (153084/153084), 152.79 MiB | 24.53 MiB/s, done.
Resolving deltas: 100% (104460/104460), done.
Checking out files: 100% (57381/57381), done.


In [3]:
# Install BeautifulSoup, this will be needed to crawl the web
#!pip3 install beautifulsoup4
#!pip3 install tqdm
#!pip3 install nltk
!pip3 install tabulate

In [1]:
# Import asyncio, this will be needed to perform asynchronous operations
import os
import asyncio
# Importing multiprocessing to assign operations to threadpools
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
# Importing this to create necessary directories
import pathlib
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import nltk
from joblib import Parallel, delayed
import json 
import math
from collections import Counter
import numpy as np

nltk.download('stopwords')
nltk.download('punkt')


import functions

''' 
if this returns an error on tabulate, install it by running the cell above.
'''

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Valentino\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Valentino\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


' \nif this returns an error on tabulate, install it by running the cell above.\n'

In [2]:
'''
Defining the amount of cores available for the process to use. If this slows your machine too much, hardcode it. 
EX: AVAILABLE_CORES = 4
'''
AVAILABLE_CORES = multiprocessing.cpu_count()
print(AVAILABLE_CORES)

# Initializing ThreadPools 
pool = ThreadPool(AVAILABLE_CORES)

12


In [14]:
def save_urls_to_txt(pages_of_urls):
    count = 0
    with open('./dataset/anime_urls.txt', 'w', encoding='utf-8') as txt_file :
        for page in pages_of_urls :
            for url in page :
                txt_file.write(url + os.linesep)
                count+=1
    print(f'written {count} lines')
    return True
            

In [5]:
# Defining pages variables based on how many pages we want to retrieve
pages = [None] * 400
pages_num = range(0,400)

# Crawl Top Animes pages #
pool.map(lambda num : functions.fetch_page(num, pages), pages_num)   
# Parallel(n_jobs=AVAILABLE_CORES)(delayed(lambda num : functions.fetch_page(num, pages))(num) for num in tqdm(pages_num))
print("Done fetching the pages!")
print("Going to fetch urls")
# Scraping all URLs present in the crawled pages #
pool.map(lambda page : functions.fetch_urls_in_page(page, pages), pages)
#Parallel(n_jobs=AVAILABLE_CORES)(delayed(lambda page : functions.fetch_urls_in_page(page, pages))(page) for page in tqdm(pages))
print("Done fetching urls")

100%|████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<?, ?it/s]

Done fetching the pages!
Going to fetch urls


Done fetching urls


In [15]:
saved_file = save_urls_to_txt(pages)
if(saved_file):
    print("Saved all URLs in ./dataset/anime_urls.txt")
else :
    print("Couldn't save URLs in txt!")

written 19139 lines
Saved all URLs in ./dataset/anime_urls.txt


In [ ]:
# Fetch animes for every requested page
'''
Here we fetch and save animes in html files. 
Starting_page defines from which page you want to resume the process. (It works as an index)

EX: 
    to start from scratch:
        starting_page = 0
    if you want to start from the 10th page:
        starting_page = 9
    if you want to set 200 as an upper bound:
        last_page = 199   
'''
starting_page = 384
last_page = len(pages)
pages_to_process = pages[starting_page:]
for i in range(0, len(pages_to_process)) : 
    functions.fetch_animes_and_save_file(pages_to_process[i], starting_page+i+1, AVAILABLE_CORES)

In [3]:
with open('./dataset/page_1/article_1.html', 'r', encoding="utf-8") as f:
    url = functions.extract_url_from_html( f.read() )
    print(url)

https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood


1.3 Parse downloaded pages
At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

Anime Name (to save as animeTitle): String.  
Anime Type (to save as animeType): String.   
Number of episode (to save as animeNumEpisode): Integer.   
Release and End Dates of anime (to save as releaseDate and endDate): Convert both release and end date into datetime format.     
Number of members (to save as animeNumMembers): Integer.    
Score (to save as animeScore): Float.   
Users (to save as animeUsers): Integer    
Rank (to save as animeRank): Integer.    
Popularity (to save as animePopularity): Integer.    
Synopsis (to save as animeDescription): String.    
Related Anime (to save as animeRelated): Extract all the related animes, but only keep unique       values and those that have a hyperlink associated to them. List of strings.        
Characters (to save as animeCharacters): List of strings.         
Voices (to save as animeVoices): List of strings.     
Staff (to save as animeStaff): Include the staff  name and their responsibility/task in a list of lists.     

In [ ]:
# test code on just one single html:
functions.extract_informations_from_anime_html('./dataset/page_1/article_1.html')

In [4]:
# find all files into directories
matches = pathlib.Path("./dataset").glob("**/*.html")
# multiprocess parsing every html into a tsv
result = Parallel(n_jobs=AVAILABLE_CORES)(delayed(functions.extract_informations_from_anime_html)(path) for path in tqdm(matches))
#pool.map(extract_informations_from_anime_html, matches)

19124it [24:57, 12.77it/s]


# 2. Search Engine   
Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.   

First, you must pre-process all the information collected for each anime by:

**Removing stopwords**   
**Removing punctuation**   
**Stemming**   
**Anything else you think it's needed**   

For this purpose, you can use the nltk library.

In [4]:
functions.preprocess_tsv('./tsv_dataset/anime_11794.tsv')

In [4]:
tsv_matches = pathlib.Path("./tsv_dataset").glob("*.tsv")
#result = pool.map(functions.preprocess_tsv, tsv_matches)
for match in tsv_matches:
    functions.preprocess_tsv(match)

2.1. Conjunctive query   
For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

2.1.1 Create your index!   
Before building the index,

- Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

In [3]:
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
functions.hydrate_vocabulary_from_files(preprocessed_files, AVAILABLE_CORES)

In [8]:
inverted_index_path = './indexes/synopsis_index.json'
vocabulary_path = './vocabulary.json'
Path("./indexes").mkdir(parents=True, exist_ok=True)
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
functions.hydrate_synopsys_inverted_index_with_given_files_and_vocabulary(inverted_index_path, preprocessed_files, vocabulary_path, AVAILABLE_CORES)

1------------starting hydrating inverted index-------------
1------------inverted index not yet existant---------------
1------------reading vocabulary----------------------------
1------------proceeding to multiprocess inputs-------------
1------------finished to multiprocess inputs---------------
1------------finished hydrating inverted index-------------
1------------dumping inverted index -----------------------
1------------dumped inverted index to path-----------------
./indexes/synopsis_index.json


In [5]:
try:
    functions.perform_query('zombie school')
except Exception as e :
    print(e)

Results for "zombie school" : 5 | elapsed time: 0.253229 seconds
| Title                                       | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|---------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

2.2) Conjunctive query & Ranking score<br />
For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contains all the words in the query.   
Sort them by their similarity with the query.   
Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.   
To solve this task, you will have to use the **tfIdf** score, and the **Cosine similarity**. The field to consider it is still the synopsis. Let's see how.

2.2.1) Inverted index
Your second Inverted Index must be of this format:

In [9]:
inverted_index_path = './indexes/tf_idf_synopsis_index.json'
idf_source_index_path = './indexes/synopsis_index.json'
vocabulary_path = './vocabulary.json'
Path("./indexes").mkdir(parents=True, exist_ok=True)
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
functions.hydrate_synopsys_inverted_index_with_given_files_and_vocabulary(
    inverted_index_path,
    preprocessed_files,
    vocabulary_path,
    AVAILABLE_CORES,
    True,
    19124,
    idf_source_index_path,
    )

1------------starting hydrating inverted index-------------
1------------inverted index not yet existant---------------
1------------reading vocabulary----------------------------
1------------proceeding to multiprocess inputs-------------
1------------finished to multiprocess inputs---------------
1------------finished hydrating inverted index-------------
1------------dumping inverted index -----------------------
1------------dumped inverted index to path-----------------
./indexes/tf_idf_synopsis_index.json


In [7]:
functions.perform_query_with_idf('saiyan race')

[0. 0. 0. ... 0. 0. 0.]
{'anime_365', 'anime_400', 'anime_1468', 'anime_1035'}
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
| Title                                                   | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|---------------------------------------------------------+----------------------------------------------